In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import os


class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [2]:
class ASLModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
                                # input: 256 x 1 x 64 x 64   
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),                                         
            nn.MaxPool2d(2, 2), # output: 32 x 32 x 32 
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.25),  # output: 128 x 16 x 16
            
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: 256 x 8 x 8
            
            
            nn.Flatten(),
            nn.Linear(256*8*8, 29)) 
            
    def forward(self, xb):
        return self.network(xb)

In [3]:
model = ASLModel()
model.load_state_dict(torch.load(r"D:\zzStudies\CSE499\GitHub\CSE499-Group01-ASL-and-BSL-Translations-using-CNN\Code\ASL\CNN0.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [4]:
import numpy as np
import cv2
#import tensorflow as tf
#mport keras

word_dict = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J', 10:'K', 11:'L', 12:'M', 13:'N', 14:'O', 15:'P', 16:'Q', 17:'R', 18:'S', 19:'T', 20:'U', 21:'V', 22:'W', 23:'X', 24:'Y', 25:'Z', 26:'del', 27:'nothing', 28:'space'}


In [5]:
def predict(input, model):
    predictions = model(input)
    _, preds = torch.max(predictions, dim=1)
    
    #print("Prediction:", preds)
    return (preds[0])

In [6]:
background = None
accumulated_weight = 0.5

def cal_accum_avg(frame, accumulated_weight):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [7]:
def is_hand_found(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    _ , thresholded = cv2.threshold(diff, threshold,255,cv2.THRESH_BINARY)
    
    # Grab the external contours for the image
    (version, _, _) = cv2.__version__.split('.')
    
    if version == '3':
        image, contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    elif version == '4':
        contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    
        
    
    if len(contours) == 0:
        return False
    else:
        return True

In [8]:
# Using cv2.rectangle() method
# Draw a rectangle with blue line borders of thickness of 2 px
#image = cv2.rectangle(image, start_point, end_point, color, thickness)

# Start coordinate, here (ROI_left, ROI_top)
# represents the top left corner of rectangle

# Ending coordinate, here (ROI-right, ROI_bottom)
# represents the bottom right corner of rectangle

ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)
num_frames = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    
    # Our operations on the frame come here
    
    #flipping the frame
    #frame = cv2.flip(frame, 1)
    
    frame_copy = frame.copy()
    
    # Draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255,0,0), 2)    
    crop_frame = frame_copy[ROI_top:ROI_bottom, ROI_right:ROI_left]
    
    crop_frame = cv2.resize(crop_frame, (64, 64))  
    #grayscale = cv2.cvtColor(crop_frame, cv2.COLOR_BGR2GRAY)
    
    resultimage = np.zeros((64, 64))
    normalizednp = cv2.normalize(crop_frame, resultimage, 0, 1, cv2.NORM_MINMAX)
    cv2.imshow('Normalized Image', normalizednp)
    #converting to tensor from np_ndarray
    x = torch.from_numpy(normalizednp)
    #converting to float32
    x = x.type(torch.float32)
    

    #print(x.view(1, 3, 64, 64))    
    
    #first dimension of tensor = batchsize, second = colour, third = height, fourth = widt
                    
    pred = predict(x.view(1, 3, 64, 64), model)
    
    
    cv2.putText(frame_copy, word_dict[pred.item()], (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
       
    
    
    # Display the frame with segmented hand
    cv2.putText(frame_copy, "Place handsign in region of interest...", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Recognition", frame_copy)
    
    # incrementing the number of frames for tracking
    num_frames += 1
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

...

tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         